In [ ]:
#import libraries
import numpy as np
import pandas
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import classification_report
import time

In [ ]:
#import data
data_small = pandas.read_csv('emails.csv')
y = data_small['Prediction'] #original
X = data_small[data_small.columns[1:-1]] #original

# 80-20 train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

In [ ]:
#preprocess

indices_to_keep=~X_train.isin([np.nan,np.inf,-np.inf]).any(1)

X_train=X_train[indices_to_keep]
y_train=y_train[indices_to_keep]

indices_to_keep_test=~X_test.isin([np.nan,np.inf,-np.inf]).any(1)

X_test=X_test[indices_to_keep_test]
y_test=y_test[indices_to_keep_test]

In [ ]:
#Neural Network to generate predictions
def predNN_2(X_train, y_train):
    clf = GridSearchCV(estimator=MLPClassifier(max_iter=50000),
          param_grid={'early_stopping' : [True, False],
                      'hidden_layer_sizes': [100, 150],
                      'activation': ['relu','tanh'],
                      'learning_rate': ['constant','adaptive'],
                      'learning_rate_init': [0.01, 0.1],
                      'solver': ['adam']},
                      cv=5,n_jobs=-1,verbose=6)
    
    # Note: GridSearchCV by default performs 5-fold CV. 
    
    print("Fitting")
    t0 = time.time()
    clf.fit(X_train, y_train)
    t1 = time.time()
    print("Optimized parameters:", clf.best_params_)
    print("Weighted validation score:", clf.best_score_)
    print("time:", t1-t0)
    return clf

In [ ]:
#Calling Neural Network for Predictions
classifier = predNN_2(X_train.values, y_train.values)
preds = classifier.predict(X_test.values)

Fitting
Fitting 5 folds for each of 32 candidates, totalling 160 fits
Optimized parameters: {'activation': 'relu', 'early_stopping': True, 'hidden_layer_sizes': 100, 'learning_rate': 'constant', 'learning_rate_init': 0.01, 'solver': 'adam'}
Weighted validation score: 0.9821104159729888
time: 1817.4925224781036


In [ ]:
with open('gridcv_adam.txt', 'w') as file:
    file.write(str(classifier.cv_results_))

In [ ]:
activation = classifier.best_params_['activation']
hidden_layer_sizes = classifier.best_params_['hidden_layer_sizes']
learning_rate = classifier.best_params_['learning_rate']
learning_rate_init = classifier.best_params_['learning_rate_init']
early_stopping = classifier.best_params_['early_stopping']

classifier = MLPClassifier(early_stopping=early_stopping, activation=activation, hidden_layer_sizes=hidden_layer_sizes, max_iter=50000, learning_rate=learning_rate, learning_rate_init=learning_rate_init)
t0 = time.time()
classifier.fit(X_train.values, y_train.values)
t1 = time.time()
print('Training time is', t1 - t0, 'seconds')

Training time is 10.122442722320557 seconds


In [ ]:
#training accuracy
train_accNN = accuracy_score(y_train.values, classifier.predict(X_train.values)) 
train_accNN

0.9973410684070583

In [ ]:
#test accuracy
test_accNN = accuracy_score(y_test.values, classifier.predict(X_test.values)) 
test_accNN

0.9729468599033816